In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName('Test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/06 18:07:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
orders = spark.read.csv('retail_db-master/orders/part-00000',
                        schema = """Order_id INT,Order_date TIMESTAMP,Customer_id INT, Status STRING""")

In [4]:
orders.show()

+--------+-------------------+-----------+---------------+
|Order_id|         Order_date|Customer_id|         Status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
|       6|2013-07-25 00:00:00|       7130|       COMPLETE|
|       7|2013-07-25 00:00:00|       4530|       COMPLETE|
|       8|2013-07-25 00:00:00|       2911|     PROCESSING|
|       9|2013-07-25 00:00:00|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSIN

In [5]:
orders.printSchema()

root
 |-- Order_id: integer (nullable = true)
 |-- Order_date: timestamp (nullable = true)
 |-- Customer_id: integer (nullable = true)
 |-- Status: string (nullable = true)



In [6]:
orders.count()

68883

In [8]:
orders.write.parquet(path = 'airlines',mode = 'overwrite',compression = 'none')

In [27]:
order_items_df = spark.read.csv('retail_db-master/order_items/part-00000')

In [31]:
order_items = spark.read.json('order_items.txt')

In [32]:
order_items.show()

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|            1|                  1|                  957|                  299.98|                  1|             299.98|
|            2|                  2|                 1073|                  199.99|                  1|             199.99|
|            3|                  2|                  502|                    50.0|                  5|              250.0|
|            4|                  2|                  403|                  129.99|                  1|             129.99|
|            5|                  4|                  897|                   24.99|                  2|              49.98|
|            6| 

In [28]:
order_items_df.show()

+---+---+----+---+------+------+
|_c0|_c1| _c2|_c3|   _c4|   _c5|
+---+---+----+---+------+------+
|  1|  1| 957|  1|299.98|299.98|
|  2|  2|1073|  1|199.99|199.99|
|  3|  2| 502|  5| 250.0|  50.0|
|  4|  2| 403|  1|129.99|129.99|
|  5|  4| 897|  2| 49.98| 24.99|
|  6|  4| 365|  5|299.95| 59.99|
|  7|  4| 502|  3| 150.0|  50.0|
|  8|  4|1014|  4|199.92| 49.98|
|  9|  5| 957|  1|299.98|299.98|
| 10|  5| 365|  5|299.95| 59.99|
| 11|  5|1014|  2| 99.96| 49.98|
| 12|  5| 957|  1|299.98|299.98|
| 13|  5| 403|  1|129.99|129.99|
| 14|  7|1073|  1|199.99|199.99|
| 15|  7| 957|  1|299.98|299.98|
| 16|  7| 926|  5| 79.95| 15.99|
| 17|  8| 365|  3|179.97| 59.99|
| 18|  8| 365|  5|299.95| 59.99|
| 19|  8|1014|  4|199.92| 49.98|
| 20|  8| 502|  1|  50.0|  50.0|
+---+---+----+---+------+------+
only showing top 20 rows



In [29]:
order_items.count()

1

In [30]:
order_items.coalesce(2).write.csv(path='Output',mode = 'overwrite',compression = 'gzip',sep = '|')

23/08/06 18:27:22 ERROR FileFormatWriter: Aborting job bf8144ba-cd49-413f-9056-6986c520b8d4.
org.apache.spark.sql.AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().
	at org.apache.spark.sql.errors.QueryCompilationErrors$.queryFromRawFilesIncludeCorruptRecordColumnError(QueryCompilationErrors.scala:2727)
	at org.apache.spark.sql.execution.datasources.json.JsonFileFormat.buildReader(JsonFileFormat.scala:113)
	at org.apache.spark.sql.execution.datasources.FileFormat.buildReaderW

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().